# Análisis y Predicción de Enfermedad Cardíaca 
**Computación en la nube — Actividad 5**

**Autor:** Tu Nombre aquí

**Fecha:** 2025-10-31 

---

## Objetivo
Realizar un análisis completo del *Heart Disease Dataset (UCI)* usando Google Colab como plataforma SaaS y aplicar algoritmos de Machine Learning (Regresión Logística y Random Forest) para predecir la presencia de enfermedad cardíaca.

---

> Este notebook está escrito en **formato académico**: incluye explicación en Markdown, código comentado, visualizaciones y conclusiones reproducibles en Google Colab.


## Índice
1. [Configuración inicial](#config)
2. [Carga de datos](#carga)
3. [Exploración de datos (EDA)](#eda)
4. [Preprocesamiento](#preproc)
5. [Modelado y evaluación](#modelado)
6. [Optimización de hiperparámetros](#opt)
7. [Conclusiones y recomendaciones](#concl)
8. [Referencias](#refs)

---

<a id='config'></a>

# 1) Configuración inicial
En esta sección instalamos (si es necesario) e importamos las librerías utilizadas. También mostramos cómo montar Google Drive para cargar y guardar archivos.

> Nota: En Colab algunas librerías ya vienen instaladas; las líneas `!pip install ...` son opcionales y se ejecutan solo si se necesita una versión específica.


In [ ]:
# Instalación de librerías (descomentarlas si las necesitas)
# !pip install seaborn plotly xgboost lightgbm

# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Opcional: montar Google Drive (descomentar en Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# 2) Carga de datos <a id='carga'></a>

Descripción breve:
- **Fuente:** UCI — Heart Disease Dataset
- **Problema:** Clasificación binaria: presencia (1) o ausencia (0) de enfermedad cardíaca.

A continuación cargamos el dataset desde la URL pública (UCI) o desde Google Drive si lo tienes guardado allí.


In [ ]:
# Cargar dataset desde UCI (si Colab tiene conexión)
# URL alternativa: usar una copia en tu Google Drive si falla
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

cols = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']

try:
    df = pd.read_csv(data_url, names=cols)
    print('Dataset cargado desde UCI. Filas:', df.shape[0])
except Exception as e:
    print('No se pudo descargar desde UCI. Asegúrate de subir el archivo a Drive y cambiar la ruta. Error:', e)
    df = pd.DataFrame()  # placeholder

# Vista rápida
df.head(10)

# 3) Exploración de datos (EDA) <a id='eda'></a>

En esta sección haremos: descripción, estadísticas, valores faltantes, distribución de variables y visualizaciones.


In [ ]:
# Información básica
print('Dimensiones:', df.shape)
print('\nTipos de datos:\n', df.dtypes)
print('\nDescripción estadística:')
display(df.describe())

# Revisar valores únicos y valores faltantes
print('\nValores únicos por columna:')
for c in df.columns:
    print(f'{c}:', df[c].unique()[:10])
    
print('\nValores faltantes por columna:')
display(df.isin(['?']).sum())  # en este dataset faltantes a veces representados por '?'
display(df.replace('?', np.nan).isna().sum())

In [ ]:
# Reemplazar '?' por NaN y convertir a numérico cuando corresponda
df = df.replace('?', np.nan)
for c in ['ca','thal']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Verificar nuevamente
display(df.info())
display(df.isna().sum())

In [ ]:
# Distribuciones y visualizaciones - al menos 5 visualizaciones
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

# 1. Histograma de edad
sns.histplot(df['age'].dropna(), bins=20, kde=True).set_title('Distribución de la edad')

# 2. Conteo de target
plt.figure(); sns.countplot(x='target', data=df).set_title('Distribución target (0 = no, 1 = yes)')

# 3. Correlación
plt.figure(figsize=(10,8)); sns.heatmap(df.corr(), annot=True, fmt='.2f').set_title('Matriz de correlación')

# 4. Boxplot: colesterol por target
plt.figure(); sns.boxplot(x='target', y='chol', data=df).set_title('Colesterol por presencia de enfermedad')

# 5. Scatter: maxHR vs age coloreado por target
plt.figure(); sns.scatterplot(x='age', y='thalach', hue='target', data=df).set_title('Frecuencia cardiaca máxima vs edad')

# 4) Preprocesamiento <a id='preproc'></a>

Pasos:
- Manejo de valores faltantes
- Codificación de variables categóricas
- Escalado de características
- División train/test


In [ ]:
# Manejo de valores faltantes: estrategia simple (imputación por media/mediana o eliminación cuando hay pocos)
print('Valores faltantes antes:')
display(df.isna().sum())

# Si hay pocas filas con NaN, podemos eliminarlas; sino imputar
missing_counts = df.isna().sum()
print('\nColumnas con missing > 0:\n', missing_counts[missing_counts>0])

# Para este notebook haremos imputación simple:
for col in df.columns:
    if df[col].dtype in [float, int] or np.issubdtype(df[col].dtype, np.number):
        df[col].fillna(df[col].median(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

print('\nValores faltantes después:')
display(df.isna().sum())

In [ ]:
# Codificación de variables categóricas (si aplica)
# En este dataset algunas variables son categóricas numéricas (cp, restecg, slope, thal)
# Convertir target a binario (en el dataset original el target puede ser 0..4 donde >0 indica enfermedad)
df['target'] = df['target'].apply(lambda x: 1 if x>0 else 0)

# Revisar distribución
display(df['target'].value_counts())

# Selección de características y separación X/y
X = df.drop('target', axis=1)
y = df['target']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print('Shapes:', X_train.shape, X_test.shape)

# 5) Modelado y evaluación <a id='modelado'></a>

Entrenaremos **Regresión Logística** como baseline y **Random Forest**. Evaluaremos con accuracy, precision, recall, f1 y AUC-ROC.


In [ ]:
# 1) Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_prob_lr = lr.predict_proba(X_test)[:,1]

print('Logistic Regression - Accuracy:', accuracy_score(y_test, y_pred_lr))
print('\nClassification Report:\n', classification_report(y_test, y_pred_lr))

# ROC AUC
print('ROC AUC:', roc_auc_score(y_test, y_prob_lr))

# 2) Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:,1]

print('\nRandom Forest - Accuracy:', accuracy_score(y_test, y_pred_rf))
print('\nClassification Report:\n', classification_report(y_test, y_pred_rf))
print('ROC AUC:', roc_auc_score(y_test, y_prob_rf))

In [ ]:
# Matriz de confusión y curvas ROC
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Confusion matrices
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_lr, cmap='viridis', values_format='d')
plt.title('LR - Confusion Matrix')
plt.subplot(1,2,2)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf, cmap='viridis', values_format='d')
plt.title('RF - Confusion Matrix')

# ROC curves
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prob_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)

plt.figure()
plt.plot(fpr_lr, tpr_lr, label=f'LogReg AUC={roc_auc_score(y_test,y_prob_lr):.2f}')
plt.plot(fpr_rf, tpr_rf, label=f'RandomForest AUC={roc_auc_score(y_test,y_prob_rf):.2f}')
plt.plot([0,1],[0,1],'--')
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.legend(); plt.title('ROC Curves')

# 6) Optimización de hiperparámetros <a id='opt'></a>

Usaremos GridSearchCV para optimizar Random Forest (ejemplo simple).

In [ ]:
# GridSearch para RandomForest (ejemplo corto para no tardar demasiado)
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=4, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)
print('Mejor parámetro:', grid.best_params_)
best_rf = grid.best_estimator_

# Evaluar mejor modelo
y_pred_best = best_rf.predict(X_test)
y_prob_best = best_rf.predict_proba(X_test)[:,1]
print('\nBest RF - Accuracy:', accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
print('ROC AUC:', roc_auc_score(y_test, y_prob_best))

# 7) Conclusiones y recomendaciones <a id='concl'></a>

**Resumen de hallazgos**:
- Resumen de resultados (compara métricas entre modelos).
- Limitaciones: tamaño del dataset, imputación simple, variables categóricas tratadas de forma básica.
- Recomendaciones: más datos, validación cruzada más amplia, probar XGBoost/LightGBM, análisis de importancia de variables, explicarabilidad (SHAP).

**Reproducibilidad**:
- Fijar `random_state` donde aplique.
- Documentar versiones de librerías y rutas a datos (colocar copia en Google Drive).

---

## Archivos a entregar
- Notebook `heart_disease_analysis_colab.ipynb`
- Carpeta `data/` con dataset original (si se agrega)
- `README.md` con instrucciones para reproducir (en el repositorio GitHub)


# 8) Referencias <a id='refs'></a>
- UCI Machine Learning Repository — Heart Disease Dataset
- Scikit-learn documentation
- Google Colab documentation

---

**Instrucciones finales:**
1. Guarda el notebook y sube a tu Google Drive o GitHub.
2. Abre en Google Colab: `File -> Upload notebook` o abrir directamente desde Drive/GitHub.
3. Ejecuta todas las celdas (Runtime -> Run all) en Colab.

**¡Listo!**